In [1]:
import torch
from torch import nn
import torch.utils.data as data
from dataset import SNPmarkersDataset
import numpy as np
from scipy.stats import pearsonr
from utils import train_DL_model, print_elapsed_time
import time
from torch.utils.data import DataLoader
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from ResGS_paper import SNPResidualDataset
from Models.ResGS import ResGSModel

In [2]:
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
DROPOUT = 0
N_LAYERS = 2
N_EPOCHS = 2
SCHEDULER_STEP_SIZE = 200
SCHEDULER_REDUCE_RATIO = 1
KERNEL_SIZE = 3
CHANNEL_FACTOR1 = 4
CHANNEL_FACTOR2 = 1.1
NFILTERS = 32

In [3]:
train_dataset = SNPmarkersDataset(mode = "local_train", skip_check=True)
validation_dataset = SNPmarkersDataset(mode = "validation", skip_check=True)
selected_phenotypes = ["ep_res"]

In [4]:
for phenotype in selected_phenotypes:
    train_dataset.set_phenotypes = phenotype
    validation_dataset.set_phenotypes = phenotype

    train_dataloader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 4)
    validation_dataloader = data.DataLoader(validation_dataset, batch_size=BATCH_SIZE, num_workers = 4)

    model = ResGSModel(NFILTERS, KERNEL_SIZE, CHANNEL_FACTOR1, CHANNEL_FACTOR2, N_LAYERS)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criteron = torch.nn.L1Loss()
    
    train_DL_model(
        model=model,
        optimizer=optimizer,
        train_dataloader=train_dataloader,
        validation_dataloader=validation_dataloader,
        criterion=criteron,
        n_epoch=N_EPOCHS,
        phenotype=phenotype,
        early_stop_n_epoch=N_EPOCHS,
        log_wandb=False,
    )

Devices detected: cpu
Model architecture : 
 ResGSModel(
  (input_block1): Res_Block(
    (block): Conv1d_BN(
      (conv): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (relu): ReLU()
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (input_block2): Res_Block(
    (block): Conv1d_BN(
      (conv): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=(1,))
      (relu): ReLU()
      (bn): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer1): Sequential(
    (0): Sequential(
      (0): Conv1d_BN(
        (conv): Conv1d(32, 128, kernel_size=(3,), stride=(2,), padding=(1,))
        (relu): ReLU()
        (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Conv1d_BN(
        (conv): Conv1d(128, 35, kernel_size=(1,), stride=(1,))
        (relu): ReLU()
        (bn): BatchNorm1d(35, eps=1e-05, momentum=0.1, affine=True

KeyboardInterrupt: 

In [ ]:
for phenotype in selected_phenotypes:
        start_time = time.time()
        train_dataset.set_phenotypes = phenotype
        validation_dataset.set_phenotypes = phenotype

        X_train = train_dataset.get_all_SNP()
        y_train = train_dataset.phenotypes[phenotype]

        X_val = validation_dataset.get_all_SNP()
        y_val = validation_dataset.phenotypes[phenotype]
        
        #Traditional model
        global bestTraditionalModel
        r_max = 0  # Record the maximum Pearson value
        for model_str in ["Ridge"]:
            if model_str == "Ridge":
                model = Ridge(random_state=2307)
            elif model_str == "support vector machine":
                model = SVR(kernel='rbf')
            elif model_str == "RandomForest":
                model = RandomForestRegressor(n_jobs=-1, random_state=2307)
            elif model_str == "GradientBoostingRegressor":
                model = GradientBoostingRegressor(random_state=2307)
            model.fit(X_train, y_train)
            y_pre = model.predict(X_val)
            r = pearsonr(y_pre, y_val).statistic
            print(f"Correlation of {r} obtained for model {model_str} in {print_elapsed_time(start_time)}.")
            if r > r_max:
                r_max = r
                bestTraditionalModel = model
            
        print(f"The best model for the phenotype {phenotype} is thus {bestTraditionalModel}")
        print(f"#########################################################################################################")

        y_train_pre = bestTraditionalModel.predict(X_train)
        y_pre = bestTraditionalModel.predict(X_val)

        print(f"Sample of original y_train:\n {y_train[0:10]}")
        print(f"Sample of original y_val: \n{y_val[0:10]}")

        y_train = (y_train - y_train_pre)
        y_val = (y_val - y_pre)

        print(f"Sample of residual y_train: \n{y_train[0:10]}")
        print(f"Sample of residual y_val: \n{y_val[0:10]}")
        

        residual_train_dataset = SNPResidualDataset(X_train.to_numpy(dtype=np.float32), y_train.to_numpy(dtype=np.float32))
        residual_validation_dataset = SNPResidualDataset(X_val.to_numpy(dtype=np.float32), y_val.to_numpy(dtype=np.float32))

        train_dataloader = DataLoader(residual_train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers = 4)
        validation_dataloader = DataLoader(residual_validation_dataset, batch_size=BATCH_SIZE, num_workers = 4)

        model = ResGSModel(NFILTERS, KERNEL_SIZE, CHANNEL_FACTOR1, CHANNEL_FACTOR2, N_LAYERS)
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
        criteron = torch.nn.L1Loss()
        
        train_DL_model(
            model=model,
            optimizer=optimizer,
            train_dataloader=train_dataloader,
            validation_dataloader=validation_dataloader,
            criterion=criteron,
            n_epoch=N_EPOCHS,
            phenotype=phenotype,
            log_wandb=False,
        )
        print(f"Computation finished in {print_elapsed_time(start_time)}")

Correlation of 0.29584881548592795 obtained for model Ridge in 0d 0h 0m 2s.
The best model for the phenotype ep_res is thus Ridge(random_state=2307)
#########################################################################################################
Sample of original y_train:
 id
BBB2024_1     2.340485
BBB2024_2    -0.029124
BBB2024_3     3.009160
BBB2024_4     3.133567
BBB2024_5     3.449609
BBB2024_6     0.394976
BBB2024_7     2.906651
BBB2024_8     1.517752
BBB2024_9    -0.086624
BBB2024_10    2.075847
Name: ep_res, dtype: float64
Sample of original y_val: 
id
BBB2024_12443    2.676483
BBB2024_12463    3.587617
BBB2024_12460    5.146492
BBB2024_12469    7.224369
BBB2024_12437    0.132438
BBB2024_12449    2.002829
BBB2024_12477    2.154037
BBB2024_14443    5.154037
BBB2024_12450    5.200983
BBB2024_12452    3.490777
Name: ep_res, dtype: float64
Sample of residual y_train: 
id
BBB2024_1     0.000225
BBB2024_2    -0.000231
BBB2024_3     0.000066
BBB2024_4     0.000142
BBB2024_5  

: 